In [ ]:
from diffusers import DiffusionPipeline
import torch

# 检查是否可以使用 MPS（Apple 的 GPU 加速）
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

print("检查 Mac mps 是否可用～～～")
print(torch.backends.mps.is_available())  # 检查 MPS 是否可用
print(torch.backends.mps.is_built())      # 检查 PyTorch 是否构建了 MPS 支持


# 从预训练模型加载 DiffusionPipeline
pipeline = DiffusionPipeline.from_pretrained(
    'model/diffsion_from_scratch.params', safety_checker=None
)


# 【工具类】获取调度器和分词器
scheduler = pipeline.scheduler # scheduler 是往图片中添加噪声的方法
tokenizer = pipeline.tokenizer

# 释放 pipeline 资源
del pipeline

# 打印设备、调度器和分词器
device, scheduler, tokenizer


In [ ]:

# 🚀🚀 生成函数, 用于测试训练好的模型, 入参是一段文本

@torch.no_grad()  # 禁用梯度计算，以节省内存和加速推理
def generate(text):
    # 词编码
    # 将输入的文本转换为编码后的张量，形状为 [1, 77]
    # 第一次编码
    pos = tokenizer(text,
                    padding='max_length',  # 对文本进行填充，使其达到最大长度
                    max_length=77,         # 最大长度为 77 个标记
                    truncation=True,       # 如果文本过长则截断
                    return_tensors='pt'    # 返回 PyTorch 张量
                    ).input_ids.to(device) # 将张量移动到指定设备（如 GPU）

    # 第二次编码 (负采样)
    # 生成一个空的文本编码作为负样本，形状为 [1, 77]
    neg = tokenizer('',
                    padding='max_length',
                    max_length=77,
                    truncation=True,
                    return_tensors='pt'
                    ).input_ids.to(device)

    # 使用编码器对正、负样本进行编码
    # 形状从 [1, 77] 转换为 [1, 77, 768]
    pos = encoder(pos)
    neg = encoder(neg)

    # 将正、负样本的编码结果拼接在一起
    # 形状从 [1, 77, 768] + [1, 77, 768] -> [2, 77, 768]
    out_encoder = torch.cat((neg, pos), dim=0)

    # 👇👇 原本是要拿一张图片的, 但我们是文生图的任务, 因此就【随机生成一张噪声图片】就好了
    # 从随机噪声开始生成 VAE 的压缩图像表示
    out_vae = torch.randn(1, 4, 64, 64, device=device)  # 初始噪声，形状为 [1, 4, 64, 64]


    # 【🚀 降噪 50 次】设置时间步长，生成过程中的步数为 50，一般从 980 递减到 0
    scheduler.set_timesteps(50, device=device)

    # 让 time 逐渐降噪, 从 999 接近于 0 
    for time in scheduler.timesteps:
        # 给图像添加噪声，用于输入 U-Net
        # 将图像和噪声拼接在一起，形状从 [1, 4, 64, 64] -> [2, 4, 64, 64]
        noise = torch.cat((out_vae, out_vae), dim=0) # 👈 正负两份 out_vae , 对应文字的 pos 跟 neg
        noise = scheduler.scale_model_input(noise, time)  # 缩放输入，以适应当前时间步长

        # 使用 U-Net 预测噪声
        # 【🔥 使用 Unet 对这张特征图进行降噪】输入带噪声的图像和文本编码，输出噪声预测，形状为 [2, 4, 64, 64]
        pred_noise = unet(out_vae=noise, out_encoder=out_encoder, time=time)

        # 从【正样本的预测】中减去【负样本的预测】，得到最终的噪声预测
        # 形状从 [2, 4, 64, 64] -> [1, 4, 64, 64]
        pred_noise = pred_noise[0] + 7.5 * (pred_noise[1] - pred_noise[0])

        # 使用 scheduler 更新图像, 重新添加噪声然后进行下一轮的降噪, 反复 50 步 （每次添加的噪声会越来越低）
        # 形状保持 [1, 4, 64, 64]
        out_vae = scheduler.step(pred_noise, time, out_vae).prev_sample


    # 【🔥 拿这张特征图放到 vae 中进行最终的解码】从 VAE 的压缩表示恢复成图像 ___________________________________________
    out_vae = 1 / 0.18215 * out_vae  # 将图像放大到原始尺度
    # 使用 VAE 解码器将图像从 [1, 4, 64, 64] 恢复到 [1, 3, 512, 512]
    image = vae.decoder(out_vae)

    # 将张量转换为图片数据
    image = image.cpu()               # 将张量移回 CPU
    image = (image + 1) / 2           # 将像素值从 [-1, 1] 范围缩放到 [0, 1] 范围
    image = image.clamp(0, 1)         # 将像素值限制在 [0, 1] 范围内
    image = image.permute(0, 2, 3, 1) # 调整维度顺序，从 [batch, channels, height, width] -> [batch, height, width, channels]
    return image.numpy()[0]           # 将张量转换为 NumPy 数组并返回第一张图片

# 调用 generate 函数生成图像，并查看生成图像的形状
generate('a drawing of a star with a jewel in the center').shape


In [ ]:
# 画图函数, 展示几张图
from matplotlib import pyplot as plt
%matplotlib inline

def show():
    texts = [
        'a drawing of a star with a jewel in the center',  #宝石海星
        'a drawing of a woman in a red cape',  #迷唇姐
        'a drawing of a dragon sitting on its hind legs',  #肥大
        'a drawing of a blue sea turtle holding a rock',  #拉普拉斯
        'a blue and white bird with its wings spread',  #急冻鸟
        'a blue and white stuffed animal sitting on top of a white surface',  #卡比兽
    ]

    images = [generate(i) for i in texts]

    plt.figure(figsize=(10, 5))
    for i in range(6):
        plt.subplot(2, 3, i + 1)
        plt.imshow(images[i])
        plt.axis('off')

    plt.show()

# 会展示【预训练的模型所画出来的图】
show()

In [ ]:
from transformers import PreTrainedModel, PretrainedConfig

# 【🔥使用我们自己训练好的模型加载数据】
#包装类
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.unet = unet.to('cpu')


#加载训练好的模型
unet = Model.from_pretrained('diffsion_from_scratch.unet').unet
unet.eval().to(device)

show()